In [ ]:
import sys

sys.path.insert(1, '../AnimationManger/')  
from AnimationManger import * 

In [ ]:

DEFAULT_COLOR = BLUE
DEFAULT_BUFF = 0.25

class CircleCluster(VGroup):
    def __init__(self, positions, color=DEFAULT_COLOR, scaler=0.25):
        super().__init__()
        self.circles = [Sphere(color=color, fill_opacity=1).scale(scaler).move_to(pos) for pos in positions]
        self.add(*self.circles)

    def connect_circles(self, arrow_configs, color=DEFAULT_COLOR):
        self.arrows = [DoubleArrow3D(self.circles[i].get_center(), self.circles[j].get_center(), color=color, **config)
                       for (i, j, config) in arrow_configs]
        self.add(*self.arrows)

    def animate_connections(self, scene, fade_in=True):
        animation = FadeIn if fade_in else FadeOut
        for arrow in self.arrows:
            scene.play(animation(arrow))

    def label_circles(self, labels, configs=None):
        if configs is None:
            configs = [{} for _ in labels]
        self.labels = [Text(label, color=BLACK, **config).scale(textScale).next_to(self.circles[i], UP*1.5)
                       for i, (label, config) in enumerate(zip(labels, configs))]
        self.add(*self.labels)

    def animate_labels(self, scene, fade_in=True):
        animation = FadeIn if fade_in else FadeOut
        for label in self.labels:
            scene.play(animation(label))
        

In [ ]:
class ArrowCreator:
    def __init__(self, create_end, create_start, text_scale):
        self.create_end = create_end
        self.create_start = create_start
        self.text_scale = text_scale

    def _create_arrows(self, group, end_func, start_func, color):
        arrows = []
        for i in range(1, 5):
            center = group[i].get_center()
            end = end_func(center)
            start = start_func(center)
            arrow = Arrow3D(start=start, end=end, color=color)
            arrows.append(arrow)
        return tuple(arrows)

    def create_top_down(self, copy, group):
        top_down_text = Text("Top-down", color=BLACK).next_to(copy, OUT).scale(self.text_scale).shift(RIGHT * 1.5, UP * 1, OUT * -1.5).rotate(2, axis=LEFT)
        top_down_arrows = self._create_arrows(group, self.create_start, self.create_end, GREEN)
        return top_down_text, top_down_arrows

    def create_bottom_up(self, copy, group):
        bottom_up_text = Text("Bottom-up", color=BLACK).next_to(copy, OUT).scale(self.text_scale).shift(RIGHT * 1.5, UP * 1, OUT * -1.5).rotate(2, axis=LEFT)
        bottom_up_arrows = self._create_arrows(group, self.create_end, self.create_start, BLUE)
        return bottom_up_text, bottom_up_arrows

In [ ]:
%%manim -ql --disable_caching --fps 10 -v WARNING processism

class processism(helpers):
    def setup(self):
        self.camera.background_color = GREY
        self.set_camera_orientation(phi=360 * DEGREES, theta=-90 * DEGREES)
        self.renderer.camera.light_source.move_to(3 *UP + 5 *LEFT)
        self.renderer.camera.ambient_light = 1.0 # Increase ambient light for less shadow
        
    def construct(self):
       
        cluster_positions = [
                            1.5 * LEFT + 2 * UP,
                            1.5 * RIGHT + 2 * UP,
                            1.5 * RIGHT + 1 * DOWN, 
                            1.5 * LEFT + 1 * DOWN]
        connections = [
            (0, 1, {}),
            (0, 3, {}),
            (1, 2, {}),
            (2, 3, {}),
            (3, 1, {}),
            (2, 0, {})
        ]

        

        labels = ["Entity 1", "Entity 2", "Entity 3", "Entity 4"]

        cluster = CircleCluster(cluster_positions)
        cluster.label_circles(labels)
        cluster.connect_circles(connections)
    


        # Create emergent_entity
        emergent_entity = Circle(color=GREEN, fill_opacity=0.2).scale(2.7).to_edge(LEFT).shift(DOWN*0.5,UP*0.3,RIGHT,OUT*-2)
        cluster.move_to(emergent_entity).shift(OUT*2,UP*0.3)
        # Create copies
        top_down_elements = VGroup(emergent_entity, *cluster.circles, *cluster.arrows)
        bottom_up_elements = top_down_elements.copy()
        bottom_up_elements.to_edge(RIGHT)
        emergent_entity.shift(LEFT*0.3)
        bottom_up_elements[0].shift(RIGHT*0.3)
        VGroup(bottom_up_elements,top_down_elements).shift(LEFT*0.4)

        # Create relation arrow
        relationArrow7 = self.create_3D_double_arrow(top_down_elements[0].get_right(), bottom_up_elements[0].get_left(), color=GREEN)

        # Create texts
        relationText = self.create_tex("A relation").next_to(cluster.arrows[0], UP)
        accumulation = self.create_tex("Accumulation").next_to(cluster.arrows[0], UP)
        Emergent = self.create_tex("Emergent entity").next_to(emergent_entity, UP)
        # Assuming create_end, create_start, and textScale are defined somewhere above
        arrowCreator = ArrowCreator(self.create_end, self.create_start, textScale)

        topDown, topDownArrows = arrowCreator.create_top_down(top_down_elements[0], top_down_elements)
        bottomUp, bottomUpArrows = arrowCreator.create_bottom_up(bottom_up_elements[0], bottom_up_elements)
        # Play animations
        self.play_animations( emergent_entity, bottom_up_elements, relationArrow7, relationText, accumulation, Emergent, topDown, topDownArrows, bottomUp, bottomUpArrows,cluster, labels, connections)

    def play_animations(self,emergent_entity, bottom_up_elements, relationArrow7, relationText, accumulation, Emergent, topDown, topDownArrows, bottomUp, bottomUpArrows,cluster, labels, connections):
        
 
        self.play(FadeIn(cluster.circles[0]))
        self.play(FadeIn(cluster.labels[0]))
        self.wait(1)
        self.play(FadeOut(cluster.labels[0]))
        self.play(FadeIn(cluster.circles[1]))
        self.play(FadeIn(cluster.arrows[0]))
        self.play(FadeIn(cluster.circles[2]), FadeIn(cluster.circles[3]))
        self.play(FadeIn(cluster.arrows[1]), FadeIn(cluster.arrows[2]), FadeIn(cluster.arrows[3]), FadeIn(cluster.arrows[4]), FadeIn(cluster.arrows[5]))
        self.play(FadeIn(accumulation))
        self.wait(1)
        self.play(FadeOut(accumulation))
        self.play(FadeIn(emergent_entity))
        self.play(FadeIn(Emergent))
        self.wait(1)
        self.play(FadeOut(Emergent))
        self.play(FadeIn(bottom_up_elements))
        self.play(FadeIn(relationArrow7))
        self.move_camera(phi=250 * DEGREES, zoom=1, run_time=2)
        self.play(FadeIn(bottomUp))
        self.play_fade_in(bottomUpArrows) 
        self.wait(1)
        self.play(FadeIn(topDown))
        self.play_fade_in(topDownArrows) 
        self.wait(1)